In [17]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_gbq
import pydata_google_auth
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import bigquery
from tqdm.notebook import tqdm
from dateutil import tz
from BigQueryTools import QueryTool
import re

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]
credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,
)

%load_ext google.cloud.bigquery
%env GCLOUD_PROJECT=nbcu-ds-sandbox-a-001

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
env: GCLOUD_PROJECT=nbcu-ds-sandbox-a-001


In [18]:
start_date = '2022-01-01'
end_date = '2023-07-31'

In [19]:
def get_monthly_report_dates(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    monthly_reports = {}

    while start <= end:
        month_year = start.strftime("%b_%Y")
        report_start_date = start.replace(day=1)
        next_month = start.replace(day=28) + timedelta(days=4)
        report_end_date = next_month - timedelta(days=next_month.day)
        monthly_reports[month_year] = {
            "report_start_date": report_start_date.strftime("%Y-%m-%d"),
            "report_end_date": report_end_date.strftime("%Y-%m-%d")
        }
        start = next_month

    return monthly_reports

def get_quarter(p_date) -> int:
    return (p_date.month - 1) // 3 + 1

def get_quarterly_report_dates(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    quarter_dates = {}

    while start_date <= end_date:
        quarter = get_quarter(start_date)
        quarter_name = 'Q{}_{}'.format(quarter, start_date.year)
        quarter_start = datetime(start_date.year, (start_date.month - 1) // 3 * 3 + 1, 1)
        quarter_end = quarter_start.replace(year=quarter_start.year + 3 * quarter // 12, month=(quarter_start.month + 3) % 12, day=1) - timedelta(days=1)
        quarter_dates[quarter_name] = {
            'report_start_date': quarter_start.strftime('%Y-%m-%d'),
            'report_end_date': quarter_end.strftime('%Y-%m-%d')
        }
        start_date = quarter_end + timedelta(days=1)

    return quarter_dates

def generate_report_variants(period_dict, suffix="report"):
    return {'_'.join([period, suffix]): {'string_format': {'report': period}, 'params': params} for period, params in period_dict.items()}

In [20]:
months = get_monthly_report_dates(start_date, end_date)
quarters = get_quarterly_report_dates(start_date, end_date)
monthly_viewing_tables = generate_report_variants(months, 'viewing')
quarterly_viewing_tables = generate_report_variants(quarters, 'viewing')
month_tables = generate_report_variants(months)
quarter_tables = generate_report_variants(quarters)
bq = QueryTool()

# Viewing Tables

In [21]:
with open('CRM_Viewing.sql') as file:
    sql_file = file.read()
    bq.add(sql_file, monthly_viewing_tables)
    bq.add(sql_file, quarterly_viewing_tables)

Replacing Jan_2022_viewing
Added: Jan_2022_viewing	 | 	Awaiting start
Replacing Feb_2022_viewing
Added: Feb_2022_viewing	 | 	Awaiting start
Replacing Mar_2022_viewing
Added: Mar_2022_viewing	 | 	Awaiting start
Replacing Apr_2022_viewing
Added: Apr_2022_viewing	 | 	Awaiting start
Replacing May_2022_viewing
Added: May_2022_viewing	 | 	Awaiting start
Replacing Jun_2022_viewing
Added: Jun_2022_viewing	 | 	Awaiting start
Replacing Jul_2022_viewing
Added: Jul_2022_viewing	 | 	Awaiting start
Added: Aug_2022_viewing	 | 	Awaiting start
Added: Sep_2022_viewing	 | 	Awaiting start
Added: Oct_2022_viewing	 | 	Awaiting start
Added: Nov_2022_viewing	 | 	Awaiting start
Added: Dec_2022_viewing	 | 	Awaiting start
Added: Jan_2023_viewing	 | 	Awaiting start
Added: Feb_2023_viewing	 | 	Awaiting start
Added: Mar_2023_viewing	 | 	Awaiting start
Added: Apr_2023_viewing	 | 	Awaiting start
Added: May_2023_viewing	 | 	Awaiting start
Added: Jun_2023_viewing	 | 	Awaiting start
Added: Jul_2023_viewing	 | 	Awaiting 

In [22]:
bq.start()

start bab28e89-22eb-4d14-8dd2-c266c2e68260
start b051b882-0073-4607-88f3-3afea47e08b3
start 7d97bb9c-799c-4e0a-8955-f0aa0e9ed16e
start e93906a6-86ab-4dea-b905-1120a5d008ad
start c60ac8ec-a57e-48da-a3d3-32b51c37b5be
start 42d689be-e39a-46fa-9f66-787735476261
start c2f85151-67ef-487f-a0fd-22ab02a0232a
start 67458caa-becd-4df1-8a6f-9da98ae8c58e
start 361cefd2-481e-45fe-bfcb-344d24e43009
start cba297f7-e3cb-445c-a41d-aa24ab3eb445
start 67f9ef67-9bd0-437b-8da4-e8e218d0b1e0
start 49ac7dcf-a621-47ce-bab7-88ab73b1de04
start 5cda9ee1-0b2d-450a-a67d-f895aef7d91c
start 41af33b7-b819-43ea-8992-cf3add70201b
start d9490f49-6642-4744-a956-971cb6ed852d
start 01845290-3b1f-4c63-823c-181916835177
start e6fcf0d4-1673-4040-839c-6fd359bc608e
start be877110-f81e-475e-9c4d-382b9dec48bc
start e7a51bfd-372a-486c-9799-87fd3e1dc1e8
start 0b551110-3332-4964-812a-8258758c050d
start eda359ec-5eb7-4047-b471-65cbf59d2e4a
start eb9e2c1a-5549-4579-94eb-3af17ec4b201
start 7967f5bd-b21a-410c-8b94-389f70865ef5
start ab2b9

In [26]:
bq.update()

Active jobs: 


Finished jobs: 
Jan_2022_viewing	 | 	b051b882-0073-4607-88f3-3afea47e08b3	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02, 2023 01:59 PM	 | 	DONE
Feb_2022_viewing	 | 	e7a51bfd-372a-486c-9799-87fd3e1dc1e8	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02, 2023 01:59 PM	 | 	DONE
Mar_2022_viewing	 | 	bab28e89-22eb-4d14-8dd2-c266c2e68260	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02, 2023 01:59 PM	 | 	DONE
Apr_2022_viewing	 | 	67f9ef67-9bd0-437b-8da4-e8e218d0b1e0	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02, 2023 01:59 PM	 | 	DONE
May_2022_viewing	 | 	cba297f7-e3cb-445c-a41d-aa24ab3eb445	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02, 2023 01:59 PM	 | 	DONE
Jun_2022_viewing	 | 	eda359ec-5eb7-4047-b471-65cbf59d2e4a	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02, 2023 01:59 PM	 | 	DONE
Q1_2022_viewing	 | 	e6fcf0d4-1673-4040-839c-6fd359bc608e	 | 	Created: Aug 02, 2023 01:44 PM	 | 	Last Checked: Aug 02